# Merge LaLiga Season CSV Files (2015-2025)

This notebook:
1. Reads all SP1 CSV files from the `data/` folder
2. Keeps only the columns: Date, HomeTeam, AwayTeam, FTHG, FTAG
3. Merges them into a single dataframe
4. Saves the result as `2015-2025.csv`

In [9]:
import pandas as pd
from pathlib import Path

# Setup paths
BASE_DIR = Path('..')
DATA_DIR = BASE_DIR / 'data'
OUTPUT_FILE = Path('.') / '2015-2025.csv'

# Columns to keep
KEEP_COLS = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG']

print(f'Reading CSV files from: {DATA_DIR}')
print(f'Will save merged result to: {OUTPUT_FILE}')

Reading CSV files from: ..\data
Will save merged result to: 2015-2025.csv


In [10]:
# Read and merge all SP1 CSV files
dfs = []
for csv_file in sorted(DATA_DIR.glob('SP1*.csv')):
    print(f'Reading {csv_file.name}')
    df = pd.read_csv(csv_file)
    # Validate required columns exist
    missing = [col for col in KEEP_COLS if col not in df.columns]
    if missing:
        print(f'WARNING: {csv_file.name} missing columns: {missing}')
        continue
    # Keep only required columns
    df = df[KEEP_COLS].copy()
    dfs.append(df)

# Merge all dataframes
if dfs:
    merged = pd.concat(dfs, axis=0, ignore_index=True)
    print(f'\nMerged {len(dfs)} files, got {len(merged)} rows')
    
    # Save merged result
    merged.to_csv(OUTPUT_FILE, index=False)
    print(f'Saved to {OUTPUT_FILE}')
else:
    print('No valid CSV files found to merge!')

Reading SP1 2015.csv
Reading SP1 2016.csv
Reading SP1 2017.csv
Reading SP1 2018.csv
Reading SP1 2019.csv
Reading SP1 2020.csv
Reading SP1 2021.csv
Reading SP1 2022.csv
Reading SP1 2023.csv
Reading SP1 2024.csv
Reading SP1 2025.csv

Merged 11 files, got 3900 rows
Saved to 2015-2025.csv


In [11]:
# Read back and display first few rows to verify
print('First 10 rows of merged data:\n')
df = pd.read_csv(OUTPUT_FILE)
print(df.head(10).to_string(index=False))

First 10 rows of merged data:

    Date   HomeTeam    AwayTeam  FTHG  FTAG
21/08/15     Malaga     Sevilla     0     0
22/08/15 Ath Madrid  Las Palmas     1     0
22/08/15    Espanol      Getafe     1     0
22/08/15  La Coruna    Sociedad     0     0
22/08/15  Vallecano    Valencia     0     0
23/08/15 Ath Bilbao   Barcelona     0     1
23/08/15      Betis  Villarreal     1     1
23/08/15    Levante       Celta     1     2
23/08/15   Sp Gijon Real Madrid     0     0
24/08/15    Granada       Eibar     1     3
